## Changing response functions
*R.A. Collenteur, University of Graz, 2021*

In this notebook the new `ChangeModel` is tested, based on the work by [Obergfjell et al. (2019)](https://ngwa.onlinelibrary.wiley.com/doi/10.1111/gwat.12891). The main idea is to apply different response functions for two different periods. As an example we look at the the groundwater levels measured near the river the Mur in Austria, where a dam was recently built. 



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pastas as ps

ps.set_log_level("ERROR")
ps.show_versions(numba=True)

## 1. Load the data

In [ ]:
prec = pd.read_csv("data_step/prec.csv", index_col=0, 
                   parse_dates=True).squeeze()
evap = pd.read_csv("data_step/evap.csv", index_col=0, 
                   parse_dates=True).squeeze()
head = pd.read_csv("data_step/head.csv", index_col=0,
                   parse_dates=True).squeeze()
river = pd.read_csv("data_step/river.csv", index_col=0, 
                   parse_dates=True).squeeze()
river -= river.min()

axes = ps.plots.series(head=head, stresses=[prec, evap, river], tmin="2000",
                       labels=["Head\n[m]", "Prec\n[mm/d]", "Evap\n[mm/d]", "River [m3/d]"])

## 2. The weighting factor

The stress is convoluted two times with different response functions. Then, a weighting function is used to add the two contributions together and compute the final contribution. 

In [ ]:
npoints = 100

tchange = 50 / npoints
t = np.linspace(0, 1, npoints)
color = plt.cm.viridis(np.linspace(0, 1, 10))

for beta, c in zip(np.linspace(-1, 1, 10), color):
    beta1 = beta * npoints
    omega = 1 / (np.exp(beta1 * (t - tchange)) + 1)
    plt.plot(omega, color=c, label="$beta$={:.2f}".format(beta))
    
plt.ylabel("$\omega$ [-]")
plt.xlabel("Time [t]")
plt.legend()

## 3. Make a model
We now make two models:

- one model where we assume the response of the heads to the river level remains the same
- and one model where the response to the river levels changes. 


In [ ]:

# Normal Model
ml = ps.Model(head, name="linear")

sm = ps.StressModel(river, ps.Exponential, name="test")
step = ps.StepModel("2012-01-01", rfunc=ps.One, name="step")

ml.add_stressmodel([sm, step])
ml.solve(report=False, tmin="2004", tmax="2018", noise=True)
ml.plots.results(figsize=(10,6));

# ChangeModel
ml2 = ps.Model(head, name="linear")

cm = ps.ChangeModel(river, ps.Exponential, ps.Exponential, name="test", tchange="2012-01-01")

ml2.add_stressmodel([cm, step])
ml2.solve(report=False, tmin="2004", tmax="2018", noise=True)
ml2.plots.results(figsize=(10,6));

The second model shows a better fit, but also the step trend changed.  

In [ ]:
print("RMSE for the first model:", ml.stats.rmse().round(2))
print("RMSE for the second model:", ml2.stats.rmse().round(2))

In [ ]:
ml2.parameters

## 4. Compare the response functions
We can also look at the response to the river before and after, 

In [ ]:
plt.plot(cm.rfunc1.step(p=ml2.parameters.loc[["test_1_A", "test_1_a"], "optimal"].values))
plt.plot(cm.rfunc2.step(p=ml2.parameters.loc[["test_2_A", "test_2_a"], "optimal"].values))

plt.legend(["Before", "After"])

## 5. Another way
We can also add the stress twice, saving one parameter that needs to be estimated.

In [ ]:
ml3 = ps.Model(head, name="linear")

river1 = river.copy()
river1.loc["2012":] = 0

river2 = river.copy()
river2.loc[:"2011"] = 0

r1 = ps.StressModel(river1, rfunc=ps.Exponential, name="river", settings="level")
r2 = ps.StressModel(river2, rfunc=ps.Exponential, name="river2", settings="level")
step = ps.StepModel("2012-01-01", rfunc=ps.One, name="step")

ml3.add_stressmodel([r1, r2, step])
ml3.solve(report=False, tmin="2004", tmax="2018", noise=True)
ml3.plots.results(figsize=(10,6));

## How do the results compare?

In [ ]:
plt.plot(cm.rfunc1.step(p=ml2.parameters.loc[["test_1_A", "test_1_a"], "optimal"].values))
plt.plot(cm.rfunc2.step(p=ml2.parameters.loc[["test_2_A", "test_2_a"], "optimal"].values))

ml3.get_step_response("river").plot()
ml3.get_step_response("river2").plot()

plt.legend(["Before", "After", "Before (method 2)", "After (method 2)"])

## References

Obergfell, C., Bakker, M. and Maas, K. (2019), Identification and Explanation of a Change in the Groundwater Regime using Time Series Analysis. Groundwater, 57: 886-894. https://doi.org/10.1111/gwat.12891